In [1]:
#| hide
from l402.core import *
from l402.payment_providers import *
from l402.utils import *

# l402-python

> 

## Usage

### Installation

Install latest from the GitHub [repository][repo]:

```sh
$ pip install git+https://github.com/Fewsats/l402-python.git
```

or from [pypi][pypi]


```sh
$ pip install l402_python
```

### Documentation

Documentation can be found hosted on this GitHub [repository][repo]'s [pages][docs]. Additionally you can find package manager specific guidelines on [conda][conda] and [pypi][pypi] respectively.

[repo]: https://github.com/Fewsats/l402-python
[docs]: https://Fewsats.github.io/l402-python/
[pypi]: https://pypi.org/project/l402-python/
[conda]: https://anaconda.org/Fewsats/l402-python

## How to use

### Server

In [2]:
#| hide
from fastapi.responses import JSONResponse
from http import HTTPStatus
from cdp import *
from fastapi import FastAPI

import httpx


In [3]:
# Create FastAPI app
app = FastAPI()

server = ServerManager(app).start()

INFO:     Started server process [47447]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


## Payment Server

In [4]:
from uuid import uuid4

In [5]:
ps = PaymentServer(
    payment_request_url="http://localhost:8000/payment_request",
    onchain_provider=create_test_wallet(fund=False),
)

In [6]:
@app.get("/offers")
def offers():
    offers_list = [Offer(
        amount=1,
        currency='USD',
        description='Purchase 1 credit for API access',
        offer_id=str(uuid4()),
        payment_methods=['onchain'], # maybe not needed because we can generate it from the payment provider
        title='1 Credit Package',
        type='one-time'
    )]

    
    offers_response = ps.create_offers(offers_list)
    return JSONResponse(
        content=offers_response.model_dump(),
        status_code=402
    )

In [7]:
@app.post("/payment_request")
async def create_payment_request(request: PaymentRequest):
    payment_request = ps.create_payment_request(**request.model_dump())
    return JSONResponse(
        content=payment_request,
        status_code=200
    )

### Client

In [8]:
w = create_test_wallet(fund=False)
c = Client(onchain_provider=CoinbaseProvider(wallet=w))

In [9]:
r1 = httpx.get("http://localhost:8000/offers")
r1.status_code, r1.json()

INFO:     127.0.0.1:50799 - "GET /offers HTTP/1.1" 402 Payment Required
INFO:     127.0.0.1:50800 - "POST /payment_request HTTP/1.1" 200 OK


(402,
 {'offers': [{'amount': 1,
    'currency': 'USD',
    'description': 'Purchase 1 credit for API access',
    'offer_id': '3731f7ea-b00e-4389-9370-fd6acedffd0d',
    'payment_methods': ['onchain'],
    'title': '1 Credit Package',
    'type': 'one-time'}],
  'payment_context_token': '34b9a1f8-422b-44c2-8063-9109f5d42e17',
  'payment_request_url': 'http://localhost:8000/payment_request',
  'version': '0.2.2'})

In [10]:
if r1.status_code == HTTPStatus.PAYMENT_REQUIRED:
    try:
        ps = c.pay(r1.json())
    except Exception as e:
        print(e)

Insufficient funds: have 0, need 1.
